In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adagrad


Inp = Input((299, 299, 3))
base_model = InceptionV3(weights='imagenet',include_top=False)

x = base_model(Inp)
x = GlobalAveragePooling2D()(x) 
x = Dense(1024,activation='relu')(x)
predictions = Dense(2,activation='softmax')(x)
model = Model(inputs=Inp, outputs=predictions)

In [ ]:
import os
import numpy as np
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

SavePath = './trainedmodel/'
ModelName = 'ocnn-x-day'
base_dir = './xxx/xxx/xxx/'
train_dir = os.path.join(base_dir, 'train/')
validation_dir = os.path.join(base_dir, 'validation/')
IMAGES_FORMAT = ['.jpg']

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 100

from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size= 32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical')

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc'])

history = model.fit(
      train_generator,
      steps_per_epoch=None,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=None)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure()
plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('./Models/Accuracy_'+ModelName+'.jpg')
plt.show()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('./Models/Loss_'+ModelName+'.jpg')
plt.show()


import csv
headers = ['Epochs','TrainingAccuracy','TrainingLoss','ValidationAccuracy','ValidationLoss']
with open('./Models/'+'TrainInformation_'+ModelName+'.csv','w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(acc)):
        if (i == 0):
            f_csv.writerow([i+1,acc[i],loss[i],val_acc[i],val_loss[i]])
        else:
            f_csv.writerow([i+1,acc[i],loss[i],val_acc[i],val_loss[i]]) 


model.save('./Models/'+ModelName+'.h5')

In [ ]:
K = len(os.listdir(validation_dir+'/'+os.listdir(validation_dir)[0])) + len(os.listdir(validation_dir+'/'+os.listdir(validation_dir)[1]))
print(K)

import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import axes
from matplotlib.font_manager import FontProperties
from sklearn.metrics import roc_curve, auc
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image

batch_size=32
validation_labels = np.zeros(shape=(K,2))
validation_generator2 = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
i = 0
for ____, labels_batch in validation_generator2:
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= K:
        break

preds = model.predict_generator(validation_generator2, steps=int(np.ceil(K/batch_size)))

import tensorflow.compat.v1 as tf

tf.compat.v1.disable_eager_execution()
x = tf.strided_slice(preds,[0,1],[K+1,2])
y = tf.strided_slice(validation_labels,[0,1],[K+1,2])
x1 = tf.strided_slice(preds,[0,0],[K+1,1])
y1 = tf.strided_slice(validation_labels,[0,0],[K+1,1])

with tf.Session() as sess:
    preds2 = sess.run(x)
    validation_labels2 = sess.run(y)
    preds3 = sess.run(x1)
    validation_labels3 = sess.run(y1)

fpr,tpr,threshold = roc_curve(validation_labels2, preds2)
roc_auc = auc(fpr,tpr)

plt.figure()
lw = 2
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.savefig(SavePath+'ROC2/ROC_Validation_'+ModelName+'.jpg')
plt.show()

headers = ['FPR','TPR']
with open(SavePath+'ROC2/ROC_Validation_'+ModelName+'.csv','w',newline='') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for i in range(len(fpr)):
        f_csv.writerow([fpr[i],tpr[i]])